In [22]:
# -*- coding: utf-8 -*-

# Created on 202208181620
# Author:    zhuoyin94 <zhuoyin94@163.com>
# Github:    https://github.com/MichaelYin1994

'''
分析特征工程结果数据。
'''

import gc
import multiprocessing as mp
import os
from functools import partial

import numpy as np
import pandas as pd
from numba import jit, njit
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

tqdm.pandas()
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 300)

In [50]:
# 载入原始数据
# *************
train_feats_df = pd.read_csv('../cached_data/train_feats_df.csv')
train_feats_df.head(20)

,kpi_id,unix_ts,label,value,window_val_sum_w300,window_val_min_w300,window_val_max_w300,window_val_mean_w300,window_val_sum_w600,window_val_min_w600,window_val_max_w600,window_val_mean_w600,window_val_sum_w1200,window_val_min_w1200,window_val_max_w1200,window_val_mean_w1200,window_val_sum_w3600,window_val_min_w3600,window_val_max_w3600,window_val_mean_w3600
0,9,1491041280000,0,181.14,181.14,181.14,181.14,181.140000,181.14,181.14,181.14,181.140000,181.14,181.14,181.14,181.140000,181.14,181.14,181.14,181.140000
1,9,1491041340000,0,173.39,354.53,173.39,181.14,177.265000,354.53,173.39,181.14,177.265000,354.53,173.39,181.14,177.265000,354.53,173.39,181.14,177.265000
2,9,1491041400000,0,180.19,534.72,173.39,181.14,178.240000,534.72,173.39,181.14,178.240000,534.72,173.39,181.14,178.240000,534.72,173.39,181.14,178.240000
3,9,1491041460000,0,186.43,721.15,173.39,186.43,180.287500,721.15,173.39,186.43,180.287500,721.15,173.39,186.43,180.287500,721.15,173.39,186.43,180.287500
4,9,1491041520000,0,181.56,902.71,173.39,186.43,180.542000,902.71,173.39,186.43,180.542000,902.71,173.39,186.43,180.542000,902.71,173.39,186.43,180.542000
5,9,1491041580000,0,178.48,1081.19,173.39,186.43,180.198333,1081.19,173.39,186.43,180.198333,1081.19,173.39,186.43,180.198333,1081.19,173.39,186.43,180.198333
6,9,1491041640000,0,179.10,1079.15,173.39,186.43,179.858333,1260.29,173.39,186.43,180.041429,1260.29,173.39,186.43,180.041429,1260.29,173.39,186.43,180.041429
7,9,1491041700000,0,181.43,1087.19,178.48,186.43,181.198333,1441.72,173.39,186.43,180.215000,1441.72,173.39,186.43,180.215000,1441.72,173.39,186.43,180.215000
8,9,1491041760000,0,184.23,1091.23,178.48,186.43,181.871667,1625.95,173.39,186.43,180.661111,1625.95,173.39,186.43,180.661111,1625.95,173.39,186.43,180.661111
9,9,1491041820000,0,178.51,1083.31,178.48,184.23,180.551667,1804.46,173.39,186.43,180.446000,1804.46,173.39,186.43,180.446000,1804.46,173.39,186.43,180.446000


In [51]:
print(train_feats_df.shape, train_feats_df['kpi_id'].nunique())

(13026, 20) 1


In [52]:
train_feats_df['kpi_id'].value_counts()

9    13026
Name: kpi_id, dtype: int64

In [53]:
train_feats_df['label'].value_counts()

0    12676
1      350
Name: label, dtype: int64